# Initialisation

In [1]:
import os
import pandas as pd
os.chdir('D:\Matières\Double_diplôme_Montreal\CIRAIG\Projet_de_maîtrise\Code\V1')
import functions_v1 as functions
pd.set_option("display.max_rows",None,"display.max_columns",None)
import pickle

# Find the data

In [2]:
# Files

## Source files
data_source='D:\Matières\Double_diplôme_Montreal\CIRAIG\Projet_de_maîtrise\Code\Source_files\main_LV.xlsx'
data_impacts_source='D:\Matières\Double_diplôme_Montreal\CIRAIG\Projet_de_maîtrise\Code\Source_files\Impacts_unitaires.xlsx'
data_pre_calculated_factors='D:\Matières\Double_diplôme_Montreal\CIRAIG\Projet_de_maîtrise\Code\Source_files\pre_calc_factors.xlsx'

## Working files

data_impacts_working_file='D:\Matières\Double_diplôme_Montreal\CIRAIG\Projet_de_maîtrise\Code\Working_files\impacts.xlsx'
general_working_file='D:\Matières\Double_diplôme_Montreal\CIRAIG\Projet_de_maîtrise\Code\Working_files\data.xlsx'
temporary_saving_file=r'D:\Matières\Double_diplôme_Montreal\CIRAIG\Projet_de_maîtrise\Code\Working_files\temporary_save_factors.xlsx'
temporary_pickle_file=r'D:\Matières\Double_diplôme_Montreal\CIRAIG\Projet_de_maîtrise\Code\Working_files\temporary_pickle'

# Matrices

## A matrix
A_1st_column='P0 : Landfill'
A_last_column='market for sand | sand | Cutoff, S - RoW avoided'
A_separation_column="P9' : Avoided standard molded objects production"
A_first_line='Tire use (km) (UF)'
A_last_line='Avoided sand (kg)'

## Demand matrix

demand_first_line=A_first_line
demand_last_line=A_last_line
demand_first_column='Final demand'
demand_last_column='Final demand'

## Bound matrix

bounds_first_line='lb'
bounds_last_line='ub'
bounds_first_column=A_1st_column
bounds_last_column=A_last_column

## Phases matrix
phases_first_line='P.Production'
phases_last_line='P.End-of-life'

## Impact matrix

Impacts_first_column=A_1st_column
Impacts_last_column=A_last_column
Impacts_separation_column=A_separation_column
Impacts_IWP_Mid_first_line="IWP Mid | Climate change, long term (kg CO2 eq (long))"
Impacts_IWP_Mid_last_line="IWP Mid | Water scarcity (m3 world-eq)"
Impacts_IWP_End_first_line="IWP End | Ecosystem quality | Climate change, ecosystem quality, long term (PDF.m2.yr)"
Impacts_IWP_End_last_line="IWP End | Human health | Human health total red (DALY)"
Impact_EF_3_0_first_line="EF 3.0 |acidification | accumulated exceedance (ae) (mol H+ eq)"
Impact_EF_3_0_last_line="EF 3.0 | water use, user deprivation potential (deprivation-weighted water consumption) (m3 depriv.)"

# Build the data file

In [3]:
# Searching for the unit impacts of the aggregated processes

#Precise the database you want to use in the variable "database"
       
functions.search_impacts(data_source,
                            data_impacts_source,
                            data_impacts_working_file,
                            Impacts_first_column,
                            Impacts_separation_column,
                            Impacts_last_column,
                            Impacts_IWP_Mid_first_line,
                            Impacts_IWP_Mid_last_line,
                            Impacts_IWP_End_first_line,
                            Impacts_IWP_End_last_line,
                            Impact_EF_3_0_first_line,
                            Impact_EF_3_0_last_line, 
                            database="EV3.8",
                            open_file="n")


 Fermer le fichier 

D:\Matières\Double_diplôme_Montreal\CIRAIG\Projet_de_maîtrise\Code\Working_files\impacts.xlsx

 s'il est ouvert. 
 La procédure prend quelques secondes.


In [4]:
# Final data file construction

data=functions.data_initialisation(data_source,
                                      data_impacts_working_file,
                                      general_working_file,
                                      A_1st_column,
                                      A_last_column,
                                      A_first_line,
                                      A_last_line,
                                      demand_first_line,
                                      demand_last_line,
                                      demand_first_column,
                                      demand_last_column,
                                      bounds_first_line,
                                      bounds_last_line,
                                      bounds_first_column,
                                      bounds_last_column, 
                                      phases_first_line,
                                      phases_last_line,
                                      open_file="n")


Lancer cette fonction après avoir formaté les données d'impacts dans 
D:\Matières\Double_diplôme_Montreal\CIRAIG\Projet_de_maîtrise\Code\Working_files\impacts.xlsx

Fermer le fichier 
D:\Matières\Double_diplôme_Montreal\CIRAIG\Projet_de_maîtrise\Code\Working_files\data.xlsx
s'il est ouvert.



# Single-objective optimisation

Provide scenarios optimized for each selected objective function.

In [5]:
# If you already have your data file

data=pd.ExcelFile(r"D:\Matières\Double_diplôme_Montreal\CIRAIG\Projet_de_maîtrise\Code\Working_files\data.xlsx")

## Optimisation

In [6]:
# Read objective functions
## The objective function has to be selected in the main file by "X" in the column "Selected scenarios". Be careful it is a big X, little x will not work.
list_objective_functions=functions.objective_choice(data_source)
list_objective_functions

['IWP Mid | Climate change, short term (kg CO2 eq (short))']

In [7]:
# Optimize the objective functions
# Return a list of scenario_class variable containing information about each scenario, factors and impacts

opt_resultats=functions.optimisation_one_obj(data,list_objective_functions,temporary_saving_file,open_file="y")

L'optimisation totale prend quelques secondes
opt. IWP Mid | Climate change, short term (kg CO2 eq (short)) has been optimized successfully
Optimization done !


## Data storage

In [8]:
# Save the precedent results into pickle form

with open(temporary_pickle_file,"wb") as output_file :
    pickle.dump(opt_resultats, output_file)


## Data retrieving

In [3]:
# Recover previously calculated data
with open(temporary_pickle_file,"rb") as input_file :
    retrieved_opt_resultats = pickle.load(input_file)

retrieved_opt_resultats

In [9]:
# If you want to add those previous results to your current working session

opt_resultats_tot=retrieved_opt_resultats+opt_resultats
opt_resultats_tot

In [4]:
opt_resultats=retrieved_opt_resultats
opt_resultats

# Result conversion to csv file

In [13]:
# Create the df containing all scenarios and associated impact values

df=functions.scenario_list_to_df(opt_resultats, transpose="y")
df

,"IWP Mid | Climate change, long term (kg CO2 eq (long))","IWP Mid | Climate change, short term (kg CO2 eq (short))",IWP Mid | Fossil and nuclear energy use (MJ deprived),IWP Mid | Freshwater acidification (kg SO2 eq),IWP Mid | Freshwater ecotoxicity (CTUe),IWP Mid | Freshwater eutrophication (kg PO4 P-lim eq),IWP Mid | Human toxicity cancer (CTUh),IWP Mid | Human toxicity non cancer (CTUh),IWP Mid | Ionizing radiations (Bq C-14 eq),"IWP Mid | Land occupation, biodiversity (m2 arable land eq .yr)","IWP Mid | Land transformation, biodiversity (m2 arable land eq)",IWP Mid | Marine eutrophication (kg N N-lim eq),IWP Mid | Mineral resources use (kg deprived),IWP Mid | Ozone Layer Depletion (kg CFC-11 eq),IWP Mid | Particulate matter formation (kg PM2.5 eq),IWP Mid | Photochemical oxidant formation (kg NMVOC eq),IWP Mid | Terrestrial acidification (kg SO2 eq),IWP Mid | Water scarcity (m3 world-eq),"IWP End | Ecosystem quality | Climate change, ecosystem quality, long term (PDF.m2.yr)","IWP End | Ecosystem quality | Climate change, ecosystem quality, short term (PDF.m2.yr)",IWP End | Ecosystem quality | Freshwater acidification (PDF.m2.yr),"IWP End | Ecosystem quality | Freshwater ecotoxicity, long term (PDF.m2.yr)","IWP End | Ecosystem quality | Freshwater ecotoxicity, short term (PDF.m2.yr)",IWP End | Ecosystem quality | Freshwater eutrophication (PDF.m2.yr),"IWP End | Ecosystem quality | Ionizing radiation, ecosystem quality (PDF.m2.yr)","IWP End | Ecosystem quality | Land occupation, biodiversity (PDF.m2.yr)","IWP End | Ecosystem quality | Land transformation, biodiversity (PDF.m2.yr)","IWP End | Ecosystem quality | Marine acidification, long term (PDF.m2.yr)","IWP End | Ecosystem quality | Marine acidification, short term (PDF.m2.yr)",IWP End | Ecosystem quality | Marine eutrophication (PDF.m2.yr),IWP End | Ecosystem quality | Terrestrial acidification (PDF.m2.yr),IWP End | Ecosystem quality | Thermally polluted water (PDF.m2.yr),"IWP End | Ecosystem quality | Water availability, freshwater ecosystem (PDF.m2.yr)","IWP End | Ecosystem quality | Water availability, terrestrial ecosystem (PDF.m2.yr)",IWP End | Ecosystem quality | TOTAL (PDF.m2.yr),IWP End | Ecosystem quality | TOTAL red (PDF.m2.yr),"IWP End |Human health | Climate change, human health, long term (DALY)","IWP End | Human health | Climate change, human health, short term (DALY)","IWP End | Human health | Human toxicity cancer, long term (DALY)","IWP End | Human health | Human toxicity cancer, short term (DALY)","IWP End | Human health | Human toxicity non-cancer, long term (DALY)","IWP End | Human health | Human toxicity non-cancer, short term (DALY)","IWP End | Human health | Ionizing radiation, human health (DALY)",IWP End | Human health | Ozone layer depletion (DALY),IWP End | Human health | Particulate matter formation (DALY),IWP End | Human health | Photochemical oxidant formation (DALY),"IWP End | Human health | Water availability, human health (DALY)",IWP End | Human health | TOTAL (DALY),IWP End | Human health | TOTAL red (DALY),EF 3.0 |acidification | accumulated exceedance (ae) (mol H+ eq),EF 3.0 | climate change | global warming potential (GWP100) (kg CO2 eq),EF 3.0 | climate change: biogenic | global warming potential (GWP100) (kg CO2 eq),"EF 3.0 | climate change: fossil, global warming potential (GWP100) (kg CO2 eq)","EF 3.0 | climate change: land use and land use change, global warming potential (GWP100) (kg CO2 eq)","EF 3.0 | ecotoxicity: freshwater, comparative toxic unit for ecosystems (CTUe)","EF 3.0 | ecotoxicity: freshwater, inorganic, comparative toxic unit for ecosystems (CTUe)","EF 3.0 | ecotoxicity: freshwater, metals, comparative toxic unit for ecosystems (CTUe)","EF 3.0 | ecotoxicity: freshwater, organics, comparative toxic unit for ecosystems (CTUe)","EF 3.0 | energy resources: non-renewable, abiotic depletion potential (ADP): fossil fuels (MJ)","EF 3.0 | eutrophication: freshwater, fraction of nutrients reaching freshwater end compartment

In [14]:
# Convert your df to a csv file

df.to_csv(r"C:\Users\Lisa\SAGESSE\data_from_Lisa\unconstrained_optimisations.csv", index=False)

# Impact manipulation

This section allows to make caculation with the impact categories to add, transform, remove... categories from the data file, before optimization

## Addition of impact categories

In [5]:
# Ajout de l'imptact Ecosystem Quality, total, reduced
nom='IWP End | Ecosystem quality | TOTAL red (PDF.m2.yr)'
methode='IWP Endpoint, added impact'
sum_list=[x for x in range(20,31+1)]   # list of the column to be summed
new_scenario_list=functions.add_new_impact_cat(opt_resultats, nom, methode, sum_list)

# Ajout de l'imptact Human Health, total, reduced
nom='IWP End | Human health | TOTAL red (DALY)'
methode='IWP Endpoint, added impact'
sum_list=[x for x in range(36,43+1)]   # list of the column to be summed
new_scenario_list=functions.add_new_impact_cat(opt_resultats, nom, methode, sum_list)



Création de la nouvelle catégorie 
IWP End | Ecosystem quality | TOTAL red (PDF.m2.yr)

Addition des valeurs des catégories d'impact
IWP End | Ecosystem quality | Freshwater acidification (PDF.m2.yr)
IWP End | Ecosystem quality | Freshwater ecotoxicity, long term (PDF.m2.yr)
IWP End | Ecosystem quality | Freshwater ecotoxicity, short term (PDF.m2.yr)
IWP End | Ecosystem quality | Freshwater eutrophication (PDF.m2.yr)
IWP End | Ecosystem quality | Ionizing radiation, ecosystem quality (PDF.m2.yr)
IWP End | Ecosystem quality | Land occupation, biodiversity (PDF.m2.yr)
IWP End | Ecosystem quality | Land transformation, biodiversity (PDF.m2.yr)
IWP End | Ecosystem quality | Marine acidification, long term (PDF.m2.yr)
IWP End | Ecosystem quality | Marine acidification, short term (PDF.m2.yr)
IWP End | Ecosystem quality | Marine eutrophication (PDF.m2.yr)
IWP End | Ecosystem quality | Terrestrial acidification (PDF.m2.yr)
IWP End | Ecosystem quality | Thermally polluted water (PDF.m2.yr)


C

In [15]:
## Scenarios impacts to excel

def extract_infos_from_scenario(self, file_to_write):

    # Impact extraction

    impact_names=self.all_impacts_parameter('name')
    values=self.all_impacts_parameter('valeur_totale')
    impacts=pd.DataFrame(values, index=impact_names, columns=[self.name])
    
    # Writing in an Excel file
    impacts.to_excel(file_to_write)

file=r"D:\Matières\Double_diplôme_Montreal\CIRAIG\Projet_de_maîtrise\Code\Dump\new_excel.xlsx"
extract_infos_from_scenario(opt_resultats[0], file)